In [1]:
import numpy as np
import scipy as sp
import math
# Define known parameters
g = 9.807  # Acceleration due to gravity in m/s^2
rho1 = 1553.6; mu1 = 65e-3; N1 = 13090; D1 = 50e-3; M1 = 0.85; P1 = 8.762e5 

H1 = P1/rho1/g; Q1 = M1/rho1
Power1 = P1*Q1/0.32403

#D2 = 50e-3; rho2 = 1286; mu2 = 33.82e-3 #60% Sucrose
#D2 = 50e-3; rho2 = 1.26895e3; mu2 = 22.35e-3 #57% Sucrose
D2 = 50e-3; rho2 = 1188.8; mu2 = 21.907e-3 #Glycerol 30 Deg, 2.4:1 By Volume
#N2 = 8000; rho2 = 1286; mu2 = 33.82e-3
#D2 = 50e-3; rho2 = 1200; mu2 = 28e-3
#D2 = 50e-3; N2 = 7000; M2 = 0.40

# Define lower and upper bounds
lb = [1000, 1.25e5, 0.01, 0]  # Lower bounds
ub = [9000, 4.25e5, 0.85, 1500]  # Upper bounds
bounds = [(low, high) for low, high in zip(lb, ub)]
#np.random.seed(100)

# Define the objective function (sum of squared residuals)
def pump_equations(x):
    N2, P2, M2, Power2 = x
    
    # Specific speed equation residual
    #F1 = (N1 * np.sqrt(M1/rho1) / (P1/(rho1*g))**(3/4)) - (N2 * np.sqrt(M2/rho2) / (P2/(rho2*g))**(3/4))
    
    # Flow coefficient equation residual
    F2 = ( M1/rho1 / (N1 * D1**3)) - (M2/rho2 / (N2 * D2**3))
    
    # Head coefficient equation residual
    F3 = ( g*P1/(rho1*g) / (N1**2 * D1**2)) - ( g*P2/(rho2*g) / (N2**2 * D2**2))
    
    # Reynolds number equation residual
    F4 = ( rho1 * N1 * D1**2 / mu1) - (rho2 * N2 * D2**2 / mu2)
    
    # Power Coefficient (based on hydraulic power)
    F5 = ( Power1 / ( rho1 * N1**3 * D1**5) ) - ( Power2 / ( rho2 * N2**3 * D2**5) )
    
    # Sum of squared residuals
    #return F1**2 + F2**2 + F3**2 + F4**2
    return [F2, F3, F4, F5]

# Normalization function for better scaling
def normalize_residuals(residuals):
    # Scale each residual by a factor that brings them into a similar range
    scales = [1, 1, 1e-3, 1]  # Example scales, adjust as needed
    return [residuals[i] / scales[i] for i in range(len(residuals))]

# Objective function for minimization (sum of squares of normalized residuals)
def objective_function(x):
    residuals = pump_equations(x)
    normalized_residuals = normalize_residuals(residuals)
    return sum([r**2 for r in normalized_residuals])


# Initial guesses for unknowns
x0 = np.array([65e-3, 4e5, 0.3, 200])

# Perform optimization using 'SLSQP' (Sequential Least Squares Programming)
#result = sp.optimize.minimize(objective_function, x0, bounds=bounds, method='trust-constr', options={'disp': True, 'maxiter': 10000, 'xtol': 1e-16, 'gtol': 1e-16})
#result = sp.optimize.differential_evolution(objective_function, bounds, disp = True, popsize=300, tol = 1e-16, maxiter = 1000)
#result = sp.optimize.root(pump_equations, x0, method='trust-constr', bounds=bounds, options={'xtol': 1e-10, 'maxiter': 2000})

# Perform global optimization using basinhopping
# Basin Hopping Methods: Powell
result = sp.optimize.basinhopping(objective_function,x0=x0,minimizer_kwargs={"method": "Powell", "bounds": bounds},niter=1000,T=0.5,stepsize=1,disp=True)

D:\anaconda3\envs\myenv\Lib\site-packages\scipy\optimize\_basinhopping.py:303: OptimizeWarning: Initial guess is not within the specified bounds
  return self.minimizer(self.func, x0, **self.kwargs)


basinhopping step 0: f 1.35525e-14
basinhopping step 1: f 4.70989e-38 trial_f 4.70989e-38 accepted 1  lowest_f 4.70989e-38
found new global minimum on step 1 with function value 4.70989e-38
basinhopping step 2: f 4.9963e-38 trial_f 4.9963e-38 accepted 1  lowest_f 4.70989e-38
basinhopping step 3: f 4.70202e-38 trial_f 4.70202e-38 accepted 1  lowest_f 4.70202e-38
found new global minimum on step 3 with function value 4.70202e-38
basinhopping step 4: f 4.99607e-38 trial_f 4.99607e-38 accepted 1  lowest_f 4.70202e-38
basinhopping step 5: f 4.70226e-38 trial_f 4.70226e-38 accepted 1  lowest_f 4.70202e-38
basinhopping step 6: f 4.99614e-38 trial_f 4.99614e-38 accepted 1  lowest_f 4.70202e-38
basinhopping step 7: f 4.70234e-38 trial_f 4.70234e-38 accepted 1  lowest_f 4.70202e-38
basinhopping step 8: f 4.71027e-38 trial_f 4.71027e-38 accepted 1  lowest_f 4.70202e-38
basinhopping step 9: f 4.70198e-38 trial_f 4.70198e-38 accepted 1  lowest_f 4.70198e-38
found new global minimum on step 9 with f

In [2]:
N2, P2, M2, Power2 = result.x[0], result.x[1], result.x[2], result.x[3]

Q2 = M2/rho2
H2 = P2/rho2/g
specific_speed_1 = N1*np.sqrt(Q1)/(P1/(rho1*g))**(3/4)
specific_speed_2 = N2*np.sqrt(Q2)/(P2/(rho2*g))**(3/4)
print(f"Ns_1 = {specific_speed_1:.4f}, Ns_2 = {specific_speed_2:.4f}, Matching = {100-abs(specific_speed_1-specific_speed_2)/specific_speed_1*100} %")

flow_coeff_1 = Q1/N1/D1**3
flow_coeff_2 = Q2/N2/D2**3
print(f"Flow Coefficient Prototype = {flow_coeff_1}, Model = {flow_coeff_2}, Matching = {100-abs(flow_coeff_1-flow_coeff_2)/flow_coeff_1*100} %")

head_coeff_1 = g*P1/(rho1*g)/N1**2/D1**2
head_coeff_2 = g*P2/(rho2*g)/N2**2/D2**2
print(f"Head Coefficient Prototype = {head_coeff_1}, Model = {head_coeff_2}, Matching = {100-abs(head_coeff_1-head_coeff_2)/head_coeff_1*100} %")

Reysnold_1 = rho1*N1*D1**2/mu1
Reysnold_2 = rho2*N2*D2**2/mu2
print(f"Reynolds Number Prototype = {Reysnold_1:.4f}, Model = {Reysnold_2:.4f}, Matching = {100-abs(Reysnold_1-Reysnold_2)/Reysnold_1*100} %")

power_coeff_1 = Power1/ ( rho1 * N1**3 * D1**5)
power_coeff_2 = Power2/ ( rho2 * N2**3 * D2**5)
print(f"Power Coefficient Prototype = {power_coeff_1:.8f}, Model = {power_coeff_2:.8f}, Matching = {100-abs(power_coeff_1-power_coeff_2)/power_coeff_1*100} %")

pump_eff_1 = (P1*Q1)/Power1
pump_eff_2 = (P2*Q2)/Power2
motor_eff = 0.85
mechanical_eff = 0.99

print(f"\nPrototype's Shaft Speed is {N1:.3f} RPM\n"
      f"Prototype's Impeller Diameter is {D1*1e3:.3f} mm\n"
      f"Prototype's Mass Flow Rate is {M1:.3f} Kg/s\n"
      f"Prototype's Pressure Across Pump is {P1/1e5:.3f} Bar\n"
      f"Prototype's Pressure Head is {P1/rho1/9.807:.3f} m HAN+H2O2\n"
      f"Prototype's Pressure Head is {P1/rho1/9.807:.3f} m Testing Fluid\n"
      f"Prototype's Pressure Head is {P1/1000/9.807:.3f} m Water\n"
      f"Prototype's Volumetric Flow Rate is {Q1*1000:.4f} L/s\n"
      f"Prototype's Fluid's Viscosity is {mu1*1000:.4f} mPa-s\n"
      f"Prototype's Fluid's Density is {rho1:.4f} kg/m^3\n"
      f"Prototype's Hydraulic Power = {Q1*P1:.3f} W\n"
      f"Prototype's Shaft Power = {Power1/mechanical_eff:.3f} W\n"
      f"Prototype's Shaft Torque = {Power1/mechanical_eff/(N1*2*math.pi/60):.3f} N-m\n"
      f"Prototype's Efficiency = {pump_eff_1*100:.3f} %\n"
      f"Prototype's Electrical Power = {Power1/mechanical_eff/motor_eff:.3f} W\n")

print(f"\nModel's Shaft Speed is {N2:.3f} RPM\n"
      f"Model's Impeller Diameter is {D2*1e3:.3f} mm\n"
      f"Model's Mass Flow Rate is {M2:.3f} Kg/s\n"
      f"Model's Pressure Across Pump is {P2/1e5:.3f} Bar\n"
      f"Model's Pressure Head is {P2/rho1/9.807:.3f} m HAN+H2O2\n"
      f"Model's Pressure Head is {P2/rho2/9.807:.3f} m Testing Fluid\n"
      f"Model's Pressure Head is {P2/1000/9.807:.3f} m Water\n"
      f"Model's Volumetric Flow Rate is {Q2*1000:.4f} L/s\n"
      f"Model's Fluid's Dynamic Viscosity is {mu2*1000:.4f} mPa-s\n"
      f"Model's Fluid's Kinematic Viscosity is {mu2*10**6/rho2:.4f} mPa-s\n"
      f"Model's Fluid's Density is {rho2:.4f} kg/m^3\n"
      f"Model's Hydraulic Power = {Q2*P2:.3f} W\n"
      f"Model's Shaft Power = {Power2/mechanical_eff:.3f} W\n"
      f"Model's Shaft Torque = {Power2/mechanical_eff/(N2*2*math.pi/60):.3f} N-m\n"
      f"Model's Efficiency = {pump_eff_2*100:.3f} %\n"
      f"Model's Electrical Power = {Power2/mechanical_eff/motor_eff:.3f} W\n")

Ns_1 = 14.6617, Ns_2 = 14.6617, Matching = 99.99999999999996 %
Flow Coefficient Prototype = 0.0003343721160404991, Model = 0.0003343721160404991, Matching = 100.0 %
Head Coefficient Prototype = 0.0013165723636124806, Model = 0.0013165723636124804, Matching = 99.99999999999999 %
Reynolds Number Prototype = 782177.8462, Model = 782177.8462, Matching = 100.0 %
Power Coefficient Prototype = 0.00000136, Model = 0.00000136, Matching = 100.0 %

Prototype's Shaft Speed is 13090.000 RPM
Prototype's Impeller Diameter is 50.000 mm
Prototype's Mass Flow Rate is 0.850 Kg/s
Prototype's Pressure Across Pump is 8.762 Bar
Prototype's Pressure Head is 57.508 m HAN+H2O2
Prototype's Pressure Head is 57.508 m Testing Fluid
Prototype's Pressure Head is 89.344 m Water
Prototype's Volumetric Flow Rate is 0.5471 L/s
Prototype's Fluid's Viscosity is 65.0000 mPa-s
Prototype's Fluid's Density is 1553.6000 kg/m^3
Prototype's Hydraulic Power = 479.383 W
Prototype's Shaft Power = 1494.385 W
Prototype's Shaft Torque 